In [636]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from catboost import CatBoostRegressor
#from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
from scipy import stats
from sklearn.model_selection import GridSearchCV
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [637]:
def list_directory_tree_with_os_walk(starting_directory):
    for root, directories, files in os.walk(starting_directory):
        print(f"Directory: {root}")
        for file in files:
            print(f"  File: {file}")

list_directory_tree_with_os_walk('.')

Directory: .
  File: my_first_submission.csv
  File: .DS_Store
  File: test.csv
  File: Readme.md
  File: catBoost.ipynb
  File: sample_submission.csv
  File: read_files.ipynb
Directory: ./A
  File: X_train_observed.parquet
  File: train_targets.parquet
  File: X_train_estimated.parquet
  File: X_test_estimated.parquet
Directory: ./C
  File: X_train_observed.parquet
  File: train_targets.parquet
  File: X_train_estimated.parquet
  File: X_test_estimated.parquet
Directory: ./B
  File: X_train_observed.parquet
  File: train_targets.parquet
  File: X_train_estimated.parquet
  File: X_test_estimated.parquet
Directory: ./catboost_info
  File: learn_error.tsv
  File: .DS_Store
  File: test_error.tsv
  File: time_left.tsv
  File: catboost_training.json
Directory: ./catboost_info/learn
  File: events.out.tfevents
Directory: ./catboost_info/test
  File: events.out.tfevents
Directory: ./catboost_info/tmp
  File: cat_feature_index.eaa4b21a-9ccafede-c0d84f1d-2cb9343f.tmp
  File: cat_feature_index.

In [638]:
# Load your data
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

#interpolation and moving avarage, test this again with new model
'''# Set the index to the time column
train_a.set_index('time', inplace=True)
train_b.set_index('time', inplace=True)
train_c.set_index('time', inplace=True)

# Replace zero values with NaN
train_a.replace(0, np.nan, inplace=True)
train_b.replace(0, np.nan, inplace=True)
train_c.replace(0, np.nan, inplace=True)

# Perform time-based interpolation
train_a.interpolate(method='time', inplace=True)
train_b.interpolate(method='time', inplace=True)
train_c.interpolate(method='time', inplace=True)

# Define the window size for the moving average
window_size = 3

# Compute the moving average
train_a_smoothed = train_a.rolling(window=window_size).mean()
train_b_smoothed = train_b.rolling(window=window_size).mean()
train_c_smoothed = train_c.rolling(window=window_size).mean()

# Assuming all datasets have the same features
features = [col for col in train_a.columns]

for feature_name in features:
    fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
    
    # Plotting for dataset A
    train_a[feature_name].plot(ax=axs[0], title=f'Train A - {feature_name}')
    train_a_smoothed[feature_name].plot(ax=axs[0], linestyle='--', label=f'{window_size}-Day Moving Average')
    
    # Plotting for dataset B
    train_b[feature_name].plot(ax=axs[1], title=f'Train B - {feature_name}')
    train_b_smoothed[feature_name].plot(ax=axs[1], linestyle='--', label=f'{window_size}-Day Moving Average')
    
    # Plotting for dataset C
    train_c[feature_name].plot(ax=axs[2], title=f'Train C - {feature_name}')
    train_c_smoothed[feature_name].plot(ax=axs[2], linestyle='--', label=f'{window_size}-Day Moving Average')
    
    # Adding legends
    axs[0].legend()
    axs[1].legend()
    axs[2].legend()
    
    plt.tight_layout()
    plt.show()

# Replace zero values with NaN
train_a_smoothed.replace(np.nan,0, inplace=True)
train_b_smoothed.replace(np.nan,0, inplace=True)
train_c_smoothed.replace(np.nan,0, inplace=True)

train_a.replace( np.nan,0, inplace=True)
train_b.replace(np.nan,0, inplace=True)
train_c.replace( np.nan,0, inplace=True)'''

"# Set the index to the time column\ntrain_a.set_index('time', inplace=True)\ntrain_b.set_index('time', inplace=True)\ntrain_c.set_index('time', inplace=True)\n\n# Replace zero values with NaN\ntrain_a.replace(0, np.nan, inplace=True)\ntrain_b.replace(0, np.nan, inplace=True)\ntrain_c.replace(0, np.nan, inplace=True)\n\n# Perform time-based interpolation\ntrain_a.interpolate(method='time', inplace=True)\ntrain_b.interpolate(method='time', inplace=True)\ntrain_c.interpolate(method='time', inplace=True)\n\n# Define the window size for the moving average\nwindow_size = 3\n\n# Compute the moving average\ntrain_a_smoothed = train_a.rolling(window=window_size).mean()\ntrain_b_smoothed = train_b.rolling(window=window_size).mean()\ntrain_c_smoothed = train_c.rolling(window=window_size).mean()\n\n# Assuming all datasets have the same features\nfeatures = [col for col in train_a.columns]\n\nfor feature_name in features:\n    fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)\n    \n   

In [639]:
#Værmelding, ulike parameter fra meteomatics

X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

print(X_train_estimated_a.nunique())
'''okey, can you make a code in python that test Feature Transformation: Try transforming these features (e.g., logarithmic, polynomial transformations) to see if they can be made more predictive.
Feature Interaction: Create interaction terms between features. Sometimes, the combination of two features can be more predictive than each feature on its own. on our data, do some diffrent transformations and try to feature interact all possibilities and give me back the best feature eng'''

date_calc                           184
date_forecast                     17576
absolute_humidity_2m:gm3             79
air_density_2m:kgm3                 162
ceiling_height_agl:m              10832
clear_sky_energy_1h:J              6863
clear_sky_rad:W                    3270
cloud_base_agl:m                  11280
dew_or_rime:idx                       3
dew_point_2m:K                      275
diffuse_rad:W                      1798
diffuse_rad_1h:J                   6868
direct_rad:W                       1968
direct_rad_1h:J                    5543
effective_cloud_cover:p            1001
elevation:m                           1
fresh_snow_12h:cm                    84
fresh_snow_1h:cm                     22
fresh_snow_24h:cm                   127
fresh_snow_3h:cm                     41
fresh_snow_6h:cm                     59
is_day:idx                            2
is_in_shadow:idx                      2
msl_pressure:hPa                    704
precip_5min:mm                       23


'okey, can you make a code in python that test Feature Transformation: Try transforming these features (e.g., logarithmic, polynomial transformations) to see if they can be made more predictive.\nFeature Interaction: Create interaction terms between features. Sometimes, the combination of two features can be more predictive than each feature on its own. on our data, do some diffrent transformations and try to feature interact all possibilities and give me back the best feature eng'

In [640]:
#Observert vær

X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')


In [641]:
#Værmelding. Må matche med X_test_observed. Men dette er frem i tid. 

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')


In [642]:
#Merging estimated and observed data with the power output, but we might need to investigate if there is some differece in observed weather and weather forcast
# Combine estimated and observed data for location A
X_train_a = pd.concat([X_train_estimated_a, X_train_observed_a])

# Merge features and target data for location A
merged_data_a = pd.merge(X_train_a, train_a, left_on='date_forecast', right_on='time')


# Combine estimated and observed data for location A
X_train_b = pd.concat([X_train_estimated_b, X_train_observed_b])

# Merge features and target data for location A
merged_data_b = pd.merge(X_train_b, train_b, left_on='date_forecast', right_on='time')

# Now, calculate the correlation matrix
correlation_matrix_b = merged_data_b.corr()

# You might want to focus on the correlations with the target variable
correlations_with_target_b = correlation_matrix_b['pv_measurement'].sort_values(ascending=False)
#print(correlations_with_target_b)

# Combine estimated and observed data for location A
X_train_c = pd.concat([X_train_estimated_c, X_train_observed_c])

# Merge features and target data for location A
merged_data_c = pd.merge(X_train_c, train_c, left_on='date_forecast', right_on='time')



In [643]:
#Here we are calculating the lineral correlation of each feature to the power output
# Now, calculate the correlation matrix
correlation_matrix_a = merged_data_a.corr()

# You might want to focus on the correlations with the target variable
correlations_with_target_a = correlation_matrix_a['pv_measurement'].sort_values(ascending=False)
#print(correlations_with_target_a)

# Now, calculate the correlation matrix
correlation_matrix_b = merged_data_b.corr()

# You might want to focus on the correlations with the target variable
correlations_with_target_b = correlation_matrix_b['pv_measurement'].sort_values(ascending=False)
#print(correlations_with_target_b)

# Now, calculate the correlation matrix
correlation_matrix_c = merged_data_c.corr()

# You might want to focus on the correlations with the target variable
correlations_with_target_c = correlation_matrix_c['pv_measurement'].sort_values(ascending=False)
#print(correlations_with_target_c)


# Create a DataFrame to hold the correlation values
correlation_comparison = pd.DataFrame({
    'Feature': correlations_with_target_a.index,
    'Location_A': correlations_with_target_a.values,
    'Location_B': correlations_with_target_b.reindex(correlations_with_target_a.index).values,
    'Location_C': correlations_with_target_c.reindex(correlations_with_target_a.index).values
})

# Calculate the absolute difference in correlation values across locations
correlation_comparison['Max_Diff'] = correlation_comparison.apply(
    lambda row: max(row['Location_A'], row['Location_B'], row['Location_C']) - 
                min(row['Location_A'], row['Location_B'], row['Location_C']), axis=1)


# Sort the DataFrame based on the difference in correlation values
correlation_comparison = correlation_comparison.sort_values(by='Max_Diff', ascending=False)

# Print the DataFrame
print(correlation_comparison)





                           Feature  Location_A  Location_B  Location_C  \
12                    visibility:m    0.228371    0.121831    0.151019   
23                     prob_rime:p   -0.041333   -0.021046   -0.113571   
9                      t_1000hPa:K    0.335902    0.416585    0.427087   
11        absolute_humidity_2m:gm3    0.236673    0.323513    0.319422   
10                  dew_point_2m:K    0.245950    0.323798    0.325013   
26                 dew_or_rime:idx   -0.057400   -0.024646    0.018164   
45                is_in_shadow:idx   -0.576389   -0.510066   -0.584903   
22                            time   -0.033503   -0.107539   -0.052976   
21                   date_forecast   -0.033503   -0.107539   -0.052976   
40             total_cloud_cover:p   -0.187265   -0.142365   -0.117943   
41         effective_cloud_cover:p   -0.216672   -0.171663   -0.147640   
43             air_density_2m:kgm3   -0.356402   -0.367909   -0.421306   
4            clear_sky_energy_1h:J    

In [644]:
#Selecting features with the highest absolute values

'''df = pd.DataFrame(correlation_comparison)

# Calculate the mean of the absolute values across the three locations
df['mean_abs_value'] = df[['Location_A', 'Location_B', 'Location_C']].abs().mean(axis=1)

# Sort the dataframe by the mean_abs_value in descending order
sorted_df = df.sort_values(by='mean_abs_value', ascending=False)

# Extract the feature names from the sorted dataframe
selected_features = sorted_df['Feature'].tolist()

print(selected_features)'''

"df = pd.DataFrame(correlation_comparison)\n\n# Calculate the mean of the absolute values across the three locations\ndf['mean_abs_value'] = df[['Location_A', 'Location_B', 'Location_C']].abs().mean(axis=1)\n\n# Sort the dataframe by the mean_abs_value in descending order\nsorted_df = df.sort_values(by='mean_abs_value', ascending=False)\n\n# Extract the feature names from the sorted dataframe\nselected_features = sorted_df['Feature'].tolist()\n\nprint(selected_features)"

In [645]:
# Plotting a single feature
'''fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
feature_name = 'absolute_humidity_2m:gm3'
X_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='red')
X_train_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='blue')
X_test_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test  A', color='green')

X_train_observed_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='red')
X_train_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='blue')
X_test_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='green')

X_train_observed_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='red')
X_train_estimated_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='blue')
X_test_estimated_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2], title='Train/Test  C', color='green')'''

"fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)\nfeature_name = 'absolute_humidity_2m:gm3'\nX_train_observed_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='red')\nX_train_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test A', color='blue')\nX_test_estimated_a[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title='Train/Test  A', color='green')\n\nX_train_observed_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='red')\nX_train_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='blue')\nX_test_estimated_b[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title='Train/Test  B', color='green')\n\nX_train_observed_c[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[2

In [646]:
test = pd.read_csv('test.csv')
test


,id,time,prediction,location
0,0,2023-05-01 00:00:00,0,A
1,1,2023-05-01 01:00:00,0,A
2,2,2023-05-01 02:00:00,0,A
3,3,2023-05-01 03:00:00,0,A
4,4,2023-05-01 04:00:00,0,A
...,...,...,...,...
2155,2155,2023-07-03 19:00:00,0,C
2156,2156,2023-07-03 20:00:00,0,C
2157,2157,2023-07-03 21:00:00,0,C
2158,2158,2023-07-03 22:00:00,0,C


In [647]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

,id,prediction
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2155,2155,0
2156,2156,0
2157,2157,0
2158,2158,0


In [648]:
# Example, let the predictions be random values
test['prediction'] = np.random.rand(len(test))
print(test)
sample_submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('my_first_submission.csv', index=False)

        id                 time  prediction location
0        0  2023-05-01 00:00:00    0.420373        A
1        1  2023-05-01 01:00:00    0.572254        A
2        2  2023-05-01 02:00:00    0.965502        A
3        3  2023-05-01 03:00:00    0.596035        A
4        4  2023-05-01 04:00:00    0.443984        A
...    ...                  ...         ...      ...
2155  2155  2023-07-03 19:00:00    0.479380        C
2156  2156  2023-07-03 20:00:00    0.010849        C
2157  2157  2023-07-03 21:00:00    0.868973        C
2158  2158  2023-07-03 22:00:00    0.944916        C
2159  2159  2023-07-03 23:00:00    0.224678        C

[2160 rows x 4 columns]


In [649]:
def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [650]:
'''#Managing the data

X_train_a = pd.concat([X_train_observed_a, X_train_estimated_a])
X_train_b = pd.concat([X_train_observed_b, X_train_estimated_b])
X_train_c = pd.concat([X_train_observed_c, X_train_estimated_c])

# Split the merged data into features (X) and target (y)
merged_data = pd.merge(X_train_a, train_a, left_on='date_forecast', right_on='time')
merged_data_b = pd.merge(X_train_b, train_b, left_on='date_forecast', right_on='time')
merged_data_c = pd.merge(X_train_c, train_c, left_on='date_forecast', right_on='time')

#Adding hour, day month and year as parameters in the model from dataforecast
merged_data['hour'] = merged_data['date_forecast'].dt.hour
merged_data['day'] = merged_data['date_forecast'].dt.day
merged_data['month'] = merged_data['date_forecast'].dt.month
merged_data['year'] = merged_data['date_forecast'].dt.year

merged_data_b['hour'] = merged_data_b['date_forecast'].dt.hour
merged_data_b['day'] = merged_data_b['date_forecast'].dt.day
merged_data_b['month'] = merged_data_b['date_forecast'].dt.month
merged_data_b['year'] = merged_data_b['date_forecast'].dt.year

merged_data_c['hour'] = merged_data_c['date_forecast'].dt.hour
merged_data_c['day'] = merged_data_c['date_forecast'].dt.day
merged_data_c['month'] = merged_data_c['date_forecast'].dt.month
merged_data_c['year'] = merged_data_c['date_forecast'].dt.year

#Splitting into X and Y values
y = merged_data['pv_measurement']
X = merged_data.drop(columns=['pv_measurement', 'date_forecast', 'date_calc', 'time'])

y_b = merged_data_b['pv_measurement']
X_b = merged_data_b.drop(columns=['pv_measurement', 'date_forecast', 'date_calc','time'])

y_c = merged_data_c['pv_measurement']
X_c = merged_data_c.drop(columns=['pv_measurement', 'date_forecast', 'date_calc','time'])

#Same but for the test estimated set
X_test_estimated_a['hour'] = X_test_estimated_a['date_forecast'].dt.hour
X_test_estimated_a['day'] = X_test_estimated_a['date_forecast'].dt.day
X_test_estimated_a['month'] = X_test_estimated_a['date_forecast'].dt.month
X_test_estimated_a['year'] = X_test_estimated_a['date_forecast'].dt.year

X_test_estimated_b['hour'] = X_test_estimated_b['date_forecast'].dt.hour
X_test_estimated_b['day'] = X_test_estimated_b['date_forecast'].dt.day
X_test_estimated_b['month'] = X_test_estimated_b['date_forecast'].dt.month
X_test_estimated_b['year'] = X_test_estimated_b['date_forecast'].dt.year

X_test_estimated_c['hour'] = X_test_estimated_c['date_forecast'].dt.hour
X_test_estimated_c['day'] = X_test_estimated_c['date_forecast'].dt.day
X_test_estimated_c['month'] = X_test_estimated_c['date_forecast'].dt.month
X_test_estimated_c['year'] = X_test_estimated_c['date_forecast'].dt.year

#Dropping dates
X_test_a = X_test_estimated_a.drop(columns=[ 'date_calc', 'date_forecast'])
X_test_b = X_test_estimated_b.drop(columns=[ 'date_calc', 'date_forecast']) 
X_test_c = X_test_estimated_c.drop(columns=[  'date_calc', 'date_forecast'])'''

"#Managing the data\n\nX_train_a = pd.concat([X_train_observed_a, X_train_estimated_a])\nX_train_b = pd.concat([X_train_observed_b, X_train_estimated_b])\nX_train_c = pd.concat([X_train_observed_c, X_train_estimated_c])\n\n# Split the merged data into features (X) and target (y)\nmerged_data = pd.merge(X_train_a, train_a, left_on='date_forecast', right_on='time')\nmerged_data_b = pd.merge(X_train_b, train_b, left_on='date_forecast', right_on='time')\nmerged_data_c = pd.merge(X_train_c, train_c, left_on='date_forecast', right_on='time')\n\n#Adding hour, day month and year as parameters in the model from dataforecast\nmerged_data['hour'] = merged_data['date_forecast'].dt.hour\nmerged_data['day'] = merged_data['date_forecast'].dt.day\nmerged_data['month'] = merged_data['date_forecast'].dt.month\nmerged_data['year'] = merged_data['date_forecast'].dt.year\n\nmerged_data_b['hour'] = merged_data_b['date_forecast'].dt.hour\nmerged_data_b['day'] = merged_data_b['date_forecast'].dt.day\nmerged_d

In [651]:
#Improved manageing the train data

import pandas as pd
import numpy as np

# 1. Preprocessing: Merging, feature extraction and splitting
def preprocess_data(X_train_observed, X_train_estimated, train):
    # Merging observed and estimated
    X_train = pd.concat([X_train_observed, X_train_estimated])
    
    # Merging with train data
    merged_data = pd.merge(X_train, train, left_on='date_forecast', right_on='time')
    columns_to_extract = ['date_calc', 'date_forecast']
    for column_name in columns_to_extract:
        merged_data[f'{column_name}_hour'] = merged_data[column_name].dt.hour
        merged_data[f'{column_name}_day'] = merged_data[column_name].dt.day
        merged_data[f'{column_name}_month'] = merged_data[column_name].dt.month
        merged_data[f'{column_name}_year'] = merged_data[column_name].dt.year
        merged_data = merged_data.drop(columns=[column_name])
    # Extracting date-related features
    
    # Splitting into X and y
    y = merged_data['pv_measurement']
    X = merged_data.drop(columns=['pv_measurement', 'time'])
    
    return X, y

# 2. Data processing: NaN replacement, feature conversion and adding location column
def process_dataset(X_data, y_data, location_name, features_to_convert):
    # Replace NaN values with 0
    X_data.replace(np.nan, 0, inplace=True)
    y_data.replace(np.nan, 0, inplace=True)
    
    # Add location column
    X_data['location'] = location_name
    
    # Convert specified features to int type
    for feature in features_to_convert:
        X_data[feature] = X_data[feature].astype(int)
    
    return X_data, y_data

# 3. Encode cyclical features: Du tenker sikkert "hvorfor cyclical features?". Fordi chattern sa dette: Time cyclicality should also be addressed. Time features like hour, day, and month are cyclic in nature. Consider converting them into cyclic features
def encode_cyclical(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col] / max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col] / max_val)
    data = data.drop(col, axis=1)
    return data

# Apply the functions
X, y = preprocess_data(X_train_observed_a, X_train_estimated_a, train_a)
X_b, y_b = preprocess_data(X_train_observed_b, X_train_estimated_b, train_b)
X_c, y_c = preprocess_data(X_train_observed_c, X_train_estimated_c, train_c)

features_to_convert = ['is_day:idx','is_in_shadow:idx']


X = encode_cyclical(X, 'date_forecast_hour', 24)
X = encode_cyclical(X, 'date_forecast_day', 30.5)  # Average days in a month
X = encode_cyclical(X, 'date_forecast_month', 12)

X_b = encode_cyclical(X_b, 'date_forecast_hour', 24)
X_b = encode_cyclical(X_b, 'date_forecast_day', 30.5)
X_b = encode_cyclical(X_b, 'date_forecast_month', 12)

X_c = encode_cyclical(X_c, 'date_forecast_hour', 24)
X_c = encode_cyclical(X_c, 'date_forecast_day', 30.5)
X_c = encode_cyclical(X_c, 'date_forecast_month', 12)


#Select only some features
X, y = process_dataset(X, y, 'A', features_to_convert)
X_b, y_b = process_dataset(X_b, y_b, 'B', features_to_convert)
X_c, y_c = process_dataset(X_c, y_c, 'C', features_to_convert)

#score174:
selected_features = ['is_in_shadow:idx', 'clear_sky_energy_1h:J', 'is_day:idx', 'direct_rad_1h:J', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad_1h:J', 'diffuse_rad:W', 'sun_elevation:d']

#alle features
#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'air_density_2m:kgm3', 'relative_humidity_1000hPa:p', 'dew_point_2m:K', 'absolute_humidity_2m:gm3', 'wind_speed_v_10m:ms', 'pressure_100m:hPa', 'msl_pressure:hPa', 'pressure_50m:hPa', 'effective_cloud_cover:p', 'sfc_pressure:hPa', 'visibility:m', 'ceiling_height_agl:m', 'cloud_base_agl:m', 'total_cloud_cover:p', 'snow_water:kgm2', 'super_cooled_liquid_water:kgm2', 'wind_speed_10m:ms', 'date_calc', 'fresh_snow_24h:cm', 'snow_depth:cm', 'precip_type_5min:idx', 'fresh_snow_12h:cm', 'time', 'date_forecast', 'fresh_snow_6h:cm', 'prob_rime:p', 'fresh_snow_3h:cm', 'rain_water:kgm2', 'precip_5min:mm', 'sun_azimuth:d', 'wind_speed_u_10m:ms', 'fresh_snow_1h:cm', 'dew_or_rime:idx', 'snow_melt_10min:mm', 'wind_speed_w_1000hPa:ms', 'snow_drift:idx', 'elevation:m', 'snow_density:kgm3']

#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'location']


'''X['location'] = 'A'
X_b['location'] = 'B'
X_c['location'] = 'C'''

X = X[selected_features]
X_b = X_b[selected_features]
X_c = X_c[selected_features]


# Convert float64 columns to float32
def convert_to_float32(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

X = convert_to_float32(X)
X_b = convert_to_float32(X_b)
X_c = convert_to_float32(X_c)
'''
X, y = process_dataset(X, y, 'A', features_to_convert)
X_b, y_b = process_dataset(X_b, y_b, 'B', features_to_convert)
X_c, y_c = process_dataset(X_c, y_c, 'C', features_to_convert)'''


"\nX, y = process_dataset(X, y, 'A', features_to_convert)\nX_b, y_b = process_dataset(X_b, y_b, 'B', features_to_convert)\nX_c, y_c = process_dataset(X_c, y_c, 'C', features_to_convert)"

In [652]:
#transformation, we need to figure this out
# Apply the Yeo-Johnson transformation
'''y_transformed, lambda_param = stats.yeojohnson(y)
X_transformed = X.apply(lambda col: stats.yeojohnson(col)[0]) # Apply the transformation to each feature
# Convert the transformed data back to DataFrames
y_transformed = pd.Series(y_transformed, name=y.name)
X_transformed = pd.DataFrame(X_transformed, columns=X.columns)'''

# Apply the Yeo-Johnson transformation
'''y_transformed_b, lambda_param_b = stats.yeojohnson(y_b)
X_transformed_b = X_b.apply(lambda col: stats.yeojohnson(col)[0])  # Apply the transformation to each feature
# Convert the transformed data back to DataFrames
y_transformed_b = pd.Series(y_transformed_b, name=y_b.name)
X_transformed_b = pd.DataFrame(X_transformed_b, columns=X_b.columns)'''

# Apply the Yeo-Johnson transformation
'''y_transformed_c, lambda_param_c = stats.yeojohnson(y_c)
X_transformed_c = X_c.apply(lambda col: stats.yeojohnson(col)[0])  # Apply the transformation to each feature
# Convert the transformed data back to DataFrames
y_transformed_c = pd.Series(y_transformed_c, name=y_c.name)
X_transformed_c = pd.DataFrame(X_transformed_c, columns=X_c.columns)'''
#X = X.drop(columns=['date_forecast'])  # Drop the original datetime column

'y_transformed_c, lambda_param_c = stats.yeojohnson(y_c)\nX_transformed_c = X_c.apply(lambda col: stats.yeojohnson(col)[0])  # Apply the transformation to each feature\n# Convert the transformed data back to DataFrames\ny_transformed_c = pd.Series(y_transformed_c, name=y_c.name)\nX_transformed_c = pd.DataFrame(X_transformed_c, columns=X_c.columns)'

In [653]:
#helping polts for transformation
# Plot the distribution of the target variable before and after the transformation
'''sns.histplot(y, kde=True, label='Original', ax=axs[0])
sns.histplot(y, kde=True, label='Transformed', ax=axs[0])
axs[0].legend()
axs[0].set_title(f'{"A"}')

sns.histplot(y, kde=True, label='Original', ax=axs[1])
sns.histplot(y_transformed, kde=True, label='Transformed', ax=axs[1])
axs[1].legend()
axs[1].set_title(f'{"B"}')

sns.histplot(y, kde=True, label='Original', ax=axs[2])
sns.histplot(y_transformed, kde=True, label='Transformed', ax=axs[2])
axs[2].legend()
axs[2].set_title(f'{"C"}')
plt.legend()
plt.show()'''

'sns.histplot(y, kde=True, label=\'Original\', ax=axs[0])\nsns.histplot(y, kde=True, label=\'Transformed\', ax=axs[0])\naxs[0].legend()\naxs[0].set_title(f\'{"A"}\')\n\nsns.histplot(y, kde=True, label=\'Original\', ax=axs[1])\nsns.histplot(y_transformed, kde=True, label=\'Transformed\', ax=axs[1])\naxs[1].legend()\naxs[1].set_title(f\'{"B"}\')\n\nsns.histplot(y, kde=True, label=\'Original\', ax=axs[2])\nsns.histplot(y_transformed, kde=True, label=\'Transformed\', ax=axs[2])\naxs[2].legend()\naxs[2].set_title(f\'{"C"}\')\nplt.legend()\nplt.show()'

In [654]:
#apply log transform, need to figure this out
'''print(y_df,"y_df")
#log transform
y_df['pv_measurement'] = y_df['pv_measurement'].apply(lambda x: np.log(x + 1))
yb_df['pv_measurement'] = yb_df['pv_measurement'].apply(lambda x: np.log(x + 1))
yc_df['pv_measurement'] = yc_df['pv_measurement'].apply(lambda x: np.log(x + 1))
print(y_df, "log")'''



'print(y_df,"y_df")\n#log transform\ny_df[\'pv_measurement\'] = y_df[\'pv_measurement\'].apply(lambda x: np.log(x + 1))\nyb_df[\'pv_measurement\'] = yb_df[\'pv_measurement\'].apply(lambda x: np.log(x + 1))\nyc_df[\'pv_measurement\'] = yc_df[\'pv_measurement\'].apply(lambda x: np.log(x + 1))\nprint(y_df, "log")'

In [655]:
#Replacing Nan values with 0
X.replace(np.nan,0, inplace=True)
y.replace( np.nan,0, inplace=True)
X_b.replace(np.nan, 0,inplace=True)
y_b.replace(np.nan, 0,inplace=True)
X_c.replace(np.nan, 0,inplace=True)
y_c.replace(np.nan, 0,inplace=True)

In [656]:
#NOT IN USE
#for now, outdated as 1 model is much better than one for each location
'''cat_features = ['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx']
X['dew_or_rime:idx'] = X['dew_or_rime:idx'].astype(int)
X['is_day:idx'] = X['is_day:idx'].astype(int)
X['is_in_shadow:idx'] = X['is_in_shadow:idx'].astype(int)

X_b['dew_or_rime:idx'] = X_b['dew_or_rime:idx'].astype(int)
X_b['is_day:idx'] = X_b['is_day:idx'].astype(int)
X_b['is_in_shadow:idx'] = X_b['is_in_shadow:idx'].astype(int)

X_c['dew_or_rime:idx'] = X_c['dew_or_rime:idx'].astype(int)
X_c['is_day:idx'] = X_c['is_day:idx'].astype(int)
X_c['is_in_shadow:idx'] = X_c['is_in_shadow:idx'].astype(int)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.dtypes)

cat_model = CatBoostRegressor(
    n_estimators=2000,
    thread_count=-1,
    max_depth=4,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    od_type="Iter",
    cat_features=cat_features,
)



cat_model.fit(X_train, y_train, eval_set=(X_validation, y_validation), early_stopping_rounds=50)


X_train_b, X_validation_b, y_train_b, y_validation_b = train_test_split(X_b, y_b, test_size=0.2, random_state=42)
cat_model_b = CatBoostRegressor(
    n_estimators=2000,
    thread_count=-1,
    max_depth=4,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    od_type="Iter",
    cat_features=cat_features,
)

cat_model_b.fit(X_train_b, y_train_b, eval_set=(X_validation_b, y_validation_b), early_stopping_rounds=50)

X_train_c, X_validation_c, y_train_c, y_validation_c = train_test_split(X_c, y_c, test_size=0.2, random_state=42)
cat_model_c = CatBoostRegressor(
    n_estimators=2000,
    thread_count=-1,
    max_depth=4,
    silent=True,
    loss_function='RMSE',
    random_seed=42,
    od_type="Iter",
    cat_features=cat_features,
)

cat_model_c.fit(X_train_c, y_train_c, eval_set=(X_validation_c, y_validation_c), early_stopping_rounds=50)

print(cat_model)

param_grid = {
    'iterations': [100, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

#grid_search = GridSearchCV(CatBoostRegressor(), param_grid, cv=5, n_jobs=-1)
#grid_search.fit(X_train, y_train)'''
#prediction for 3 models, outdated as of now

'''# Make predictions
y_pred_a = cat_model.predict(X_validation)
y_pred_b = cat_model.predict(X_validation_b)
y_pred_c = cat_model.predict(X_validation_c)

# Undo the Yeo-Johnson transformation on the predictions
y_pred_original_scale_a = (y_pred_a * lambda_param + 1)**(1/lambda_param) if lambda_param != 0 else np.exp(y_pred_a)
y_pred_original_scale_b = (y_pred_b * lambda_param_b + 1)**(1/lambda_param_b) if lambda_param_b != 0 else np.exp(y_pred_b)
y_pred_original_scale_c = (y_pred_c * lambda_param_c + 1)**(1/lambda_param_c) if lambda_param_c != 0 else np.exp(y_pred_c)

# Calculate the RMSE
rmse_a = np.sqrt(mean_squared_error(y_validation, y_pred_a))
print(f'RMSE: {rmse_a}')

# Calculate the RMSE
rmse_b = np.sqrt(mean_squared_error(y_validation_b, y_pred_b))
print(f'RMSE: {rmse_b}')
# Calculate the RMSE
rmse_c = np.sqrt(mean_squared_error(y_validation_c, y_pred_c))
print(f'RMSE: {rmse_c}')
'''
'''# If you want to evaluate the model on the original scale
y_test_original_scale_a = (y_validation * lambda_param + 1)**(1/lambda_param) if lambda_param != 0 else np.exp(y_validation)
rmse_original_scale_a = np.sqrt(mean_squared_error(y_test_original_scale_a, y_pred_original_scale_a))
print(f'RMSE (original scale): {rmse_original_scale_a}')

# If you want to evaluate the model on the original scale
y_test_original_scale_b = (y_validation_b * lambda_param_b + 1)**(1/lambda_param_b) if lambda_param_b != 0 else np.exp(y_validation_b)
rmse_original_scale_b = np.sqrt(mean_squared_error(y_test_original_scale_b, y_pred_original_scale_b))
print(f'RMSE (original scale): {rmse_original_scale_b}')
# If you want to evaluate the model on the original scale
y_test_original_scale_c = (y_pred_original_scale_c * lambda_param_c + 1)**(1/lambda_param_c) if lambda_param_c != 0 else np.exp(y_pred_original_scale_c)
rmse_original_scale_c = np.sqrt(mean_squared_error(y_test_original_scale_c, y_pred_original_scale_c))
print(f'RMSE (original scale): {rmse_original_scale_c}')'''
'''RMSE: 314.78827508863765
RMSE: 997.4962753104166
RMSE: 1059.2958848252413'''
#NOT IN USE
#prediction of the 3 catboost model, outdated
#print(X_test_estimated_a.dtypes)


'''X_test_a = X_test_estimated_a.drop(columns=[ 'date_calc', 'date_forecast'])#,  'elevation:m', 'wind_speed_w_1000hPa:ms','snow_drift:idx', 'snow_density:kgm3'])
X_test_b = X_test_estimated_b.drop(columns=[ 'date_calc', 'date_forecast']) #, 'elevation:m', 'wind_speed_w_1000hPa:ms','snow_drift:idx', 'snow_density:kgm3'])
X_test_c = X_test_estimated_c.drop(columns=[  'date_calc', 'date_forecast'])# , 'elevation:m', 'wind_speed_w_1000hPa:ms','snow_drift:idx', 'snow_density:kgm3'])


# List of features to convert to categorical
features_to_convert = ['dew_or_rime:idx', 'is_day:idx','is_in_shadow:idx']

# Convert features for X
for feature in features_to_convert:
    X_test_a[feature] = X_test_a[feature].astype(int)

# Convert features for X_b
for feature in features_to_convert:
    X_test_b[feature] = X_test_b[feature].astype(int)

# Convert features for X_c
for feature in features_to_convert:
    X_test_c[feature] = X_test_c[feature].astype(int)
print(X_test_a.dtypes)

cat_prediction = cat_model.predict(X_test_a)

cat_prediction_b = cat_model.predict(X_test_b)

cat_prediction_c = cat_model.predict(X_test_c)
print(cat_prediction,"before")'''
#undo log transform
'''cat_prediction = np.exp(cat_prediction) - 1
cat_prediction_b = np.exp(cat_prediction_b) - 1
cat_prediction_c = np.exp(cat_prediction_c) - 1'''
#print(cat_prediction,"after")
'''print(len(cat_prediction))
print(len(cat_prediction_b))
print(len(cat_prediction_c))'''
#bruke lag features
# Set negative values to mean
'''cat_prediction[cat_prediction < 0] = 0

cat_prediction_b[cat_prediction_b < 0] = 0

cat_prediction_c[cat_prediction_c < 0] = 0'''


'''hourly_prediction_a = cat_prediction[::4]
#print(len(hourly_prediction_a))
hourly_prediction_b = cat_prediction_b[::4]
hourly_prediction_c = cat_prediction_c[::4]
num_predictions = len(hourly_prediction_a)
timestamps = pd.date_range(start='2023-05-01 00:00:00', periods=num_predictions, freq='H')
# Step 1: Create individual DataFrames for each location
df_a = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_a,
    'location': 'A'
})

df_b = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_b,
    'location': 'B'
})

df_c = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_c,
    'location': 'C'
})
#print(df_a)

# Step 2: Concatenate these DataFrames
final_df = pd.concat([df_a, df_b, df_c], ignore_index=True)

# Step 3: Reset the index to get the 'id' column
final_df.reset_index(inplace=True)
final_df.rename(columns={'index': 'id'}, inplace=True)

# Now final_df is in the desired format
#print(final_df)
#print(final_df.columns)
final_df= final_df.drop(columns=['time', 'location'])

#print(final_df)
sample_submission = sample_submission[['id']].merge(final_df[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('my_first_submission.csv', index=False)
'''


'''cat_mse = round(rmsle(y_validation, cat_prediction), 4)
print('RMSLE:', cat_mse)
plt.figure(figsize=(6, 3), dpi=200)
plt.scatter(cat_prediction, y_validation)
plt.grid()
plt.show()
# Plot feature importances

feature_importance = cat_model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(8, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X.columns)[sorted_idx])
plt.title('Feature Importance')'''

"cat_mse = round(rmsle(y_validation, cat_prediction), 4)\nprint('RMSLE:', cat_mse)\nplt.figure(figsize=(6, 3), dpi=200)\nplt.scatter(cat_prediction, y_validation)\nplt.grid()\nplt.show()\n# Plot feature importances\n\nfeature_importance = cat_model.feature_importances_\nsorted_idx = np.argsort(feature_importance)\nfig = plt.figure(figsize=(8, 6))\nplt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')\nplt.yticks(range(len(sorted_idx)), np.array(X.columns)[sorted_idx])\nplt.title('Feature Importance')"

In [657]:
#1 model instead of 3, CaaaaaaaaatBooooozzzzt

# Add location column
'''X['location'] = 'A'
X_b['location'] = 'B'
X_c['location'] = 'C'''

# List of features to convert to categorical
features_to_convert = ['is_day:idx','is_in_shadow:idx']

# Convert features for X
for feature in features_to_convert:
    X[feature] = X[feature].astype(int)

# Convert features for X_b
for feature in features_to_convert:
    X_b[feature] = X_b[feature].astype(int)

# Convert features for X_c
for feature in features_to_convert:
    X_c[feature] = X_c[feature].astype(int)

# Combine the datasets
X_combined = pd.concat([X, X_b, X_c])
y_combined = pd.concat([y, y_b, y_c])

#assert 'location' in X_combined.columns, "Location column is missing in combined dataset"


# Split into training and validation sets
X_train_combined, X_validation_combined, y_train_combined, y_validation_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

print(X_train_combined.dtypes)


is_in_shadow:idx           int64
clear_sky_energy_1h:J    float32
is_day:idx                 int64
direct_rad_1h:J          float32
clear_sky_rad:W          float32
direct_rad:W             float32
diffuse_rad_1h:J         float32
diffuse_rad:W            float32
sun_elevation:d          float32
dtype: object


In [658]:
#1 model instead of 3

#adding location to cat_features
cat_features = ['is_day:idx', 'is_in_shadow:idx']


cat_model_combined = CatBoostRegressor(
    n_estimators=2000,
    thread_count=-1,
    max_depth=4,
    loss_function='RMSE',
    random_seed=42,
    od_type="Iter",
    logging_level='Verbose',
    cat_features=cat_features
)


selected_features = ['is_in_shadow:idx', 'clear_sky_energy_1h:J', 'is_day:idx', 'direct_rad_1h:J', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad_1h:J', 'diffuse_rad:W', 'sun_elevation:d']

#selected_features = ['is_in_shadow:idx', 'clear_sky_energy_1h:J', 'is_day:idx', 'direct_rad_1h:J', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad_1h:J', 'diffuse_rad:W', 'sun_elevation:d', 'location']
#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'air_density_2m:kgm3', 'relative_humidity_1000hPa:p', 'dew_point_2m:K', 'absolute_humidity_2m:gm3', 'wind_speed_v_10m:ms']
#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'location']

print(y_train_combined.dtypes)


# Train the model with the specified feature types
cat_model_combined.fit(X_train_combined, y_train_combined, cat_features=cat_features, eval_set=(X_validation_combined, y_validation_combined))




float64
Learning rate set to 0.066279
0:	learn: 730.3541780	test: 721.5046425	best: 721.5046425 (0)	total: 10.5ms	remaining: 21.1s
1:	learn: 714.6617078	test: 705.8498702	best: 705.8498702 (1)	total: 13.5ms	remaining: 13.4s
2:	learn: 700.8212647	test: 691.9834517	best: 691.9834517 (2)	total: 17.4ms	remaining: 11.6s
3:	learn: 688.5388607	test: 679.7151880	best: 679.7151880 (3)	total: 19.9ms	remaining: 9.91s
4:	learn: 677.3486597	test: 668.5915355	best: 668.5915355 (4)	total: 22.6ms	remaining: 9s
5:	learn: 667.3512091	test: 658.6842507	best: 658.6842507 (5)	total: 24.9ms	remaining: 8.27s
6:	learn: 658.7423043	test: 650.1264549	best: 650.1264549 (6)	total: 27.2ms	remaining: 7.75s
7:	learn: 650.7623559	test: 642.1906286	best: 642.1906286 (7)	total: 29.8ms	remaining: 7.42s
8:	learn: 643.7804880	test: 635.3586071	best: 635.3586071 (8)	total: 32.2ms	remaining: 7.12s
9:	learn: 637.4261545	test: 629.0994191	best: 629.0994191 (9)	total: 34.9ms	remaining: 6.94s
10:	learn: 631.9563962	test: 623.67

In [659]:
#managing the test data

def preprocess_test_data(X):
    X = X.fillna(0)
    return X

def encode_cyclical(df, column, max_value):
    # Transforming the column into its sine and cosine components
    df[column + '_sin'] = np.sin(2 * np.pi * df[column]/max_value)
    df[column + '_cos'] = np.cos(2 * np.pi * df[column]/max_value)
    
    # Optionally, you can drop the original column; depends on your preference and model's need
    df = df.drop(column, axis=1)
    
    return df

def prepare_test_data(X_test, location_name, features_to_convert, cat_features, nan_replacement="missing_value"):
    # Add location column
    X_test['location'] = location_name
    
    # Convert specified features to int type and replace NaN values
    for feature in features_to_convert:
        X_test[feature] = X_test[feature].apply(lambda x: int(x) if not pd.isna(x) else x)

    # Replace NaN values in categorical features
    for feature in cat_features:
        X_test[feature].fillna(nan_replacement, inplace=True)
    
    return X_test


def extract_datetime_features(df, column_names):
    for column_name in column_names:
        df[f'{column_name}_hour'] = df[column_name].dt.hour
        df[f'{column_name}_day'] = df[column_name].dt.day
        df[f'{column_name}_month'] = df[column_name].dt.month
        df[f'{column_name}_year'] = df[column_name].dt.year
        df = df.drop(columns=[column_name])
    return df


# 1. Preprocess test data
X_test_a = preprocess_test_data(X_test_estimated_a)
X_test_b = preprocess_test_data(X_test_estimated_b)
X_test_c = preprocess_test_data(X_test_estimated_c)

# 2. Extract datetime features
columns_to_extract = ['date_calc', 'date_forecast']
X_test_a = extract_datetime_features(X_test_a, columns_to_extract)
X_test_b = extract_datetime_features(X_test_b, columns_to_extract)
X_test_c = extract_datetime_features(X_test_c, columns_to_extract)

#print(X_test_a.dtypes)


# 3. Cyclically encode date-related features
X_test_a = encode_cyclical(X_test_a, 'date_forecast_hour', 24)
X_test_a = encode_cyclical(X_test_a, 'date_forecast_day', 30.5)
X_test_a = encode_cyclical(X_test_a, 'date_forecast_month', 12)

X_test_b = encode_cyclical(X_test_b, 'date_forecast_hour', 24)
X_test_b = encode_cyclical(X_test_b, 'date_forecast_day', 30.5)
X_test_b = encode_cyclical(X_test_b, 'date_forecast_month', 12)

X_test_c = encode_cyclical(X_test_c, 'date_forecast_hour', 24)
X_test_c = encode_cyclical(X_test_c, 'date_forecast_day', 30.5)
X_test_c = encode_cyclical(X_test_c, 'date_forecast_month', 12)


# 4. Further prepare the test data
features_to_convert = ['is_day:idx', 'is_in_shadow:idx']
cat_features = ['is_day:idx', 'is_in_shadow:idx', 'location']

X_test_a = prepare_test_data(X_test_a, 'A', features_to_convert, cat_features)
X_test_b = prepare_test_data(X_test_b, 'B', features_to_convert, cat_features)
X_test_c = prepare_test_data(X_test_c, 'C', features_to_convert, cat_features)


# 5. Filter to selected features

selected_features = ['is_in_shadow:idx', 'clear_sky_energy_1h:J', 'is_day:idx', 'direct_rad_1h:J', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad_1h:J', 'diffuse_rad:W', 'sun_elevation:d']
#selected_features = ['clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad:W', 'total_cloud_cover:p', 'sun_elevation:d', 'is_day:idx', 'is_in_shadow:idx', 'location']
#selected_features = ['is_in_shadow:idx', 'clear_sky_energy_1h:J', 'is_day:idx', 'direct_rad_1h:J', 'clear_sky_rad:W', 'direct_rad:W', 'diffuse_rad_1h:J', 'diffuse_rad:W', 'sun_elevation:d', 'location']
#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'air_density_2m:kgm3', 'relative_humidity_1000hPa:p', 'dew_point_2m:K', 'absolute_humidity_2m:gm3', 'wind_speed_v_10m:ms', 'location']
#selected_features = ['direct_rad:W', 'direct_rad_1h:J', 'clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'sun_elevation:d', 'is_in_shadow:idx', 'is_day:idx', 't_1000hPa:K', 'location']

X_test_a = X_test_a[selected_features]
X_test_b = X_test_b[selected_features]
X_test_c = X_test_c[selected_features]


assert all(X_test_a.columns == X_train_combined.columns), "Mismatch in column names or order between X_test_a and X_train_combined"
assert all(X_test_b.columns == X_train_combined.columns), "Mismatch in column names or order between X_test_b and X_train_combined"
assert all(X_test_c.columns == X_train_combined.columns), "Mismatch in column names or order between X_test_c and X_train_combined"


# 6. Convert float64 columns to float32
def convert_to_float32(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

X_test_a = convert_to_float32(X_test_a)
X_test_b = convert_to_float32(X_test_b)
X_test_c = convert_to_float32(X_test_c)



#Testet med og uten denne, ble dårligere med..
'''features_to_convert = ['is_day:idx', 'is_in_shadow:idx']

# Convert features for X_test_a
for feature in features_to_convert:
    X_test_a[feature] = X_test_a[feature].apply(lambda x: int(x) if not pd.isna(x) else x)

# Convert features for X_test_b
for feature in features_to_convert:
    X_test_b[feature] = X_test_b[feature].apply(lambda x: int(x) if not pd.isna(x) else x)

# Convert features for X_test_c
for feature in features_to_convert:
    X_test_c[feature] = X_test_c[feature].apply(lambda x: int(x) if not pd.isna(x) else x)

nan_replacement = "missing_value"

# Replace NaN values in categorical features for X_test_a, X_test_b, and X_test_c
for feature in cat_features:
    X_test_a[feature].fillna(nan_replacement, inplace=True)
    X_test_b[feature].fillna(nan_replacement, inplace=True)
    X_test_c[feature].fillna(nan_replacement, inplace=True)

    
    # Convert float64 columns to float32
def convert_to_float32(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

X_test_a = convert_to_float32(X_test_a)
X_test_b = convert_to_float32(X_test_b)
X_test_c = convert_to_float32(X_test_c)
'''


'features_to_convert = [\'is_day:idx\', \'is_in_shadow:idx\']\n\n# Convert features for X_test_a\nfor feature in features_to_convert:\n    X_test_a[feature] = X_test_a[feature].apply(lambda x: int(x) if not pd.isna(x) else x)\n\n# Convert features for X_test_b\nfor feature in features_to_convert:\n    X_test_b[feature] = X_test_b[feature].apply(lambda x: int(x) if not pd.isna(x) else x)\n\n# Convert features for X_test_c\nfor feature in features_to_convert:\n    X_test_c[feature] = X_test_c[feature].apply(lambda x: int(x) if not pd.isna(x) else x)\n\nnan_replacement = "missing_value"\n\n# Replace NaN values in categorical features for X_test_a, X_test_b, and X_test_c\nfor feature in cat_features:\n    X_test_a[feature].fillna(nan_replacement, inplace=True)\n    X_test_b[feature].fillna(nan_replacement, inplace=True)\n    X_test_c[feature].fillna(nan_replacement, inplace=True)\n\n    \n    # Convert float64 columns to float32\ndef convert_to_float32(df):\n    for col in df.columns:\n   

In [660]:
#1 model predictions, catboost

#print(X_test_a.iloc[:, 55])
print(X_test_a.dtypes)


cat_features = ['is_day:idx', 'is_in_shadow:idx', 'location']



# Model predictions
cat_prediction_a = cat_model_combined.predict(X_test_a)
cat_prediction_b = cat_model_combined.predict(X_test_b)
cat_prediction_c = cat_model_combined.predict(X_test_c)


hourly_prediction_a = cat_prediction_a[::4]
hourly_prediction_b = cat_prediction_b[::4]
hourly_prediction_c = cat_prediction_c[::4]

num_predictions = len(hourly_prediction_a)
timestamps = pd.date_range(start='2023-05-01 00:00:00', periods=num_predictions, freq='H')

# Step 1: Create individual DataFrames for each location
df_a = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_a,
    'location': 'A'
})

df_b = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_b,
    'location': 'B'
})

df_c = pd.DataFrame({
    'time': timestamps,
    'prediction': hourly_prediction_c,
    'location': 'C'
})

# Step 2: Concatenate these DataFrames
final_df = pd.concat([df_a, df_b, df_c], ignore_index=True)

# Step 3: Reset the index to get the 'id' column
final_df.reset_index(inplace=True)
final_df.rename(columns={'index': 'id'}, inplace=True)

# Now final_df is in the desired format
#print(final_df)
#print(final_df.columns)
final_df= final_df.drop(columns=['time', 'location'])

#print(final_df)
sample_submission = sample_submission[['id']].merge(final_df[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('my_first_submission.csv', index=False)

is_in_shadow:idx           int64
clear_sky_energy_1h:J    float32
is_day:idx                 int64
direct_rad_1h:J          float32
clear_sky_rad:W          float32
direct_rad:W             float32
diffuse_rad_1h:J         float32
diffuse_rad:W            float32
sun_elevation:d          float32
dtype: object
